# SimAFM Tests

## Imports

In [ ]:
import os
import numpy as np

import AFMpy as AFM

import MDAnalysis as MDA
from MDAnalysis import transformations

## Loading Trajectory

In [ ]:
### Trajectory Paths
PSF_PATH = 'traj/POPE_3DIN.psf'
DCD_PATH = 'traj/POPE_3DIN_Aligned.dcd'

roty = transformations.rotateby(180, direction = [0,1,0], point = [0,0,0])
rotz = transformations.rotateby(-90, direction = [0,0,1], point = [0,0,0])
workflow = [roty, rotz]

cytoplasmic_universe = MDA.Universe(PSF_PATH, DCD_PATH, transformations = workflow)

## Adjustable Parameters

In [ ]:
# Tip Radius
tip_radius = 20

# Tip Half Angle
tip_theta = 18

# Scanning boundaries and shape
boundaries = ((-64, 64), (-64, 64))
scan_range = boundaries[0][1] - boundaries[0][0]
scan_shape = (64,64)

# VDW Dictionary. All Atom or CG
vdw_dict =  AFM.SimAFM.AA_VDW_Dict()

# Bookkeeping parameters
stack_code = f'AC-{int(tip_radius)}-{int(scan_range / scan_shape[0])}'
metadata = {
    'aligned': True,
    'alignment_method': 'Transmembrane Backbone RMSD Minimization',
    'Tip Radius': tip_radius,
    'Tip Theta': tip_theta,
    'Stack Code': stack_code,
    'side': 'Cytoplasmic',
    'normalized': False,
    'PDB Code': '3DIN',
    'Membrane Composition': 'POPE',
    'Trajectory File': 'POPE_3DIN_Aligned.dcd',
    'PSF File': 'POPE_3DIN.psf',
}

# Location to save stack
output_directory = 'stacks'
filename = f'{stack_code}.xz'

In [ ]:
# Create Grid
grid = AFM.SimAFM.make_grid(boundaries, scan_shape)

# Create Simulated Stack
im_stack = AFM.SimAFM.simulate_AFM2D_stack_MP(cytoplasmic_universe,
                                                'protein',
                                                'name P and prop z > 0',
                                                grid,
                                                tip_radius,
                                                tip_theta,
                                                vdw_dict,
                                                n_procs = 2)

# Make the stack object
output_stack = AFM.SimAFM.SimAFM_Stack(im_stack, grid, **metadata)

# Make the output string
out_str = os.path.join(output_directory, filename)

# Save the stack
output_stack.save_lzma(out_str)